In [1]:
print("OK")

OK


In [2]:
%pwd

'd:\\llm_project\\medical-chatbox\\research'

In [18]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\llm_project\\medical-chatbox'

In [19]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
# Extract Data from the PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf", 
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [21]:
extracted_data=load_pdf_file(data='Data/')

In [12]:
#extracted_data

In [22]:

#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [23]:
text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 5860


In [20]:
#text_chunks

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings

In [31]:
#pip install -U sentence-transformers transformers huggingface-hub

In [25]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [26]:
embeddings = download_hugging_face_embeddings()

In [27]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [34]:
#query_result

In [55]:
from dotenv import load_dotenv
load_dotenv()

True

In [56]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [30]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-m82i78z.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null,
    "_response_info": {
        "raw_headers": {
            "content-type": "application/json",
            "access-control-allow-origin": "*",
            "vary": "access-control-request-headers",
            "access-control-expose-headers": "*",
            "x-pinecone-api-version": "2025-10",
            "x-cloud-trace-context": "d3c

In [57]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [32]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [33]:
# Download existing index
from langchain_pinecone import Pinecone
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [34]:
docsearch

In [35]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [36]:
retrieved_docs = retriever.invoke("What is a Acne?")

In [37]:
retrieved_docs

[Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'creationdate': '2004-12-

In [38]:
#from langchain_openai import OpenAI
#llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
#from langchain_google_genai import ChatGoogleGenerativeAI

#llm = ChatGoogleGenerativeAI( model="gemini-1.5-flash",temperature=0.4,max_output_tokens=512)

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="llama2",
    temperature=0.4,
)

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [60]:
rag_chain = (
    {
        "context": retriever,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
)

In [ ]:
response = rag_chain.invoke("what is Acromegaly and gigantism?")
print("Response:\n")
print(response)